# Univ. of Illinois Data Mining Project on Coursera
## Task 00 - Clean and Convert Yelp Reviews from JSON
2018-09-14
loganjtravis@gmail.com (Logan Travis)

In [1]:
# Imports
import json, os
import pandas as pd

### Summary

Tasks in the Univeristy of Illinois Data Minig Project use the Yelp data set. See course page [Week 1 > Orientation > Data Set and Toolkit Acquisition](https://www.coursera.org/learn/data-mining-project/supplement/Ij7rp/data-set-and-toolkit-acquisition). That data set includes multiple JSON files but I intend to use Pandas dataframes for most of my work. Parsing the source files once and savingas a pickled dataframe will save me time and provide a consistent source for each task. This workbook parses then saves the `yelp_academic_dataset_review.json` data set.

### Get Data Set

In [2]:
# Set paths to data source and output
PATH_SOURCE = "source/yelp_dataset_challenge_academic_dataset/"
PATH_OUTPUT = "source/"

# Set review file path
PATH_SOURCE_YELP_REVIEWS = PATH_SOURCE + "yelp_academic_dataset_review.json"

# Set ouptput file path (using GZIP compression)
PATH_OUTPUT_YELP_REVIEWS = PATH_OUTPUT + "yelp_academic_dataset_review.pkl.gzip"

The Yelp data set did not provide a schema. All files use JSON formatting but the `pandas.read_json` ([link to documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_json.html)) requires an `orient` parameter to correctly parse JSON. Here are the first three lines of the review dataset:

In [3]:
# Since the data did not include a schema description, print the first 3 lines
with open(PATH_SOURCE_YELP_REVIEWS) as f:
    for i in range(3):
        print(f"Line {i:d}: {json.loads(next(f))}\n")

Line 0: {'votes': {'funny': 0, 'useful': 2, 'cool': 1}, 'user_id': 'Xqd0DzHaiyRqVH3WRG7hzg', 'review_id': '15SdjuK7DmYqUAj6rjGowg', 'stars': 5, 'date': '2007-05-17', 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.", 'type': 'review', 'business_id': 'vcNAWiLM4dR7D2nwwJ7nCA'}

Line 1: {'votes': {'funny': 0, 'useful': 2, 'cool': 0}, 'user_id': 'H1kH6QZV7Le4zqTRNxoZow', 'review_id': 'RF6UnRTtG7tWMcrO2GEoAg', 'stars': 2, 'date': '2010-03-22', 'text': "Unfortunately, the frustration of being Dr. Goldberg'

Each row is an object and they appear consistent so set `orient` to "records" and `lines` to `True`...

...rather those settings would work with a smaller dataset. The Yelp review is **huge**:

In [4]:
# Print file statistics
print(f"Size: {os.path.getsize(PATH_SOURCE_YELP_REVIEWS) >> 20:,} MB")
print(f"Lines: {sum(1 for l in open(PATH_SOURCE_YELP_REVIEWS)):,}")

Size: 997 MB
Lines: 1,125,458


Save memory by passing a generator to the `pandas.DataFrame` initializer that reads one line at a time and parses the JSON. Results:

In [5]:
# Load reviews into Pandas dataframe one line at a time
with open(PATH_SOURCE_YELP_REVIEWS) as f:
#     dfYelpReviews = pd.DataFrame(json.loads(l) for l in f if random.random() < 0.3)
    dfYelpReviews = pd.DataFrame(json.loads(l) for l in f)

In [6]:
# Print dataframe shape and head
print(f"Shape: {dfYelpReviews.shape}")
dfYelpReviews.head()

Shape: (1125458, 8)


,business_id,date,review_id,stars,text,type,user_id,votes
0,vcNAWiLM4dR7D2nwwJ7nCA,2007-05-17,15SdjuK7DmYqUAj6rjGowg,5,dr. goldberg offers everything i look for in a...,review,Xqd0DzHaiyRqVH3WRG7hzg,"{'funny': 0, 'useful': 2, 'cool': 1}"
1,vcNAWiLM4dR7D2nwwJ7nCA,2010-03-22,RF6UnRTtG7tWMcrO2GEoAg,2,"Unfortunately, the frustration of being Dr. Go...",review,H1kH6QZV7Le4zqTRNxoZow,"{'funny': 0, 'useful': 2, 'cool': 0}"
2,vcNAWiLM4dR7D2nwwJ7nCA,2012-02-14,-TsVN230RCkLYKBeLsuz7A,4,Dr. Goldberg has been my doctor for years and ...,review,zvJCcrpm2yOZrxKffwGQLA,"{'funny': 0, 'useful': 1, 'cool': 1}"
3,vcNAWiLM4dR7D2nwwJ7nCA,2012-03-02,dNocEAyUucjT371NNND41Q,4,Been going to Dr. Goldberg for over 10 years. ...,review,KBLW4wJA_fwoWmMhiHRVOA,"{'funny': 0, 'useful': 0, 'cool': 0}"
4,vcNAWiLM4dR7D2nwwJ7nCA,2012-05-15,ebcN2aqmNUuYNoyvQErgnA,4,Got a letter in the mail last week that said D...,review,zvJCcrpm2yOZrxKffwGQLA,"{'funny': 0, 'useful': 2, 'cool': 1}"


### Clean Data Set

The dataset requires some cleanup. Expanding the `votes` feature and setting `review_id` as the index:

In [7]:
# Concatenate columns from expanded `votes` JSON column to dataframe then
# delete original column.
#
# NOTE: Concatenating before chaning index because the expansion of `votes`
# maintains order by not the index.
dfYelpReviews = pd.concat([
    dfYelpReviews,
    pd.io.json.json_normalize(dfYelpReviews.votes)
], axis=1)
del dfYelpReviews["votes"]

In [8]:
# Relabel expanded columns for clarity
dfYelpReviews.rename(mapper={
    "cool": "votes_cool",
    "funny": "votes_funny",
    "useful": "votes_useful"
}, axis=1, inplace=True)

In [9]:
# Set `review_id` as index
dfYelpReviews.set_index("review_id", inplace=True)

In [10]:
# Print dataframe head after cleanup
dfYelpReviews.head()

,business_id,date,stars,text,type,user_id,votes_cool,votes_funny,votes_useful
review_id,,,,,,,,,
15SdjuK7DmYqUAj6rjGowg,vcNAWiLM4dR7D2nwwJ7nCA,2007-05-17,5,dr. goldberg offers everything i look for in a...,review,Xqd0DzHaiyRqVH3WRG7hzg,1,0,2
RF6UnRTtG7tWMcrO2GEoAg,vcNAWiLM4dR7D2nwwJ7nCA,2010-03-22,2,"Unfortunately, the frustration of being Dr. Go...",review,H1kH6QZV7Le4zqTRNxoZow,0,0,2
-TsVN230RCkLYKBeLsuz7A,vcNAWiLM4dR7D2nwwJ7nCA,2012-02-14,4,Dr. Goldberg has been my doctor for years and ...,review,zvJCcrpm2yOZrxKffwGQLA,1,0,1
dNocEAyUucjT371NNND41Q,vcNAWiLM4dR7D2nwwJ7nCA,2012-03-02,4,Been going to Dr. Goldberg for over 10 years. ...,review,KBLW4wJA_fwoWmMhiHRVOA,0,0,0
ebcN2aqmNUuYNoyvQErgnA,vcNAWiLM4dR7D2nwwJ7nCA,2012-05-15,4,Got a letter in the mail last week that said D...,review,zvJCcrpm2yOZrxKffwGQLA,1,0,2


Set appropriate data types:

In [11]:
# Set `date` column data type to datetime
dfYelpReviews.date = pd.to_datetime(dfYelpReviews.date, format="%Y-%m-%d")

In [12]:
# set `type` column data type to category
dfYelpReviews.type = dfYelpReviews.type.astype("category")

In [13]:
# Print data types
dfYelpReviews.dtypes

business_id             object
date            datetime64[ns]
stars                    int64
text                    object
type                  category
user_id                 object
votes_cool               int64
votes_funny              int64
votes_useful             int64
dtype: object

### Save Pickled Data Set

Save the clean data set as a GZIPped pickle to "/source/{filename}.pkl.gzip". Not much smaller:

In [14]:
dfYelpReviews.to_pickle(PATH_OUTPUT_YELP_REVIEWS)

In [15]:
print(f"Size: {os.path.getsize(PATH_OUTPUT_YELP_REVIEWS) >> 20:,} MB")

Size: 877 MB
